In [4]:
path = 'ICE-Nigeria-txt-and-xml-files_version_Nov_03_2015/ice-nig/txt - without speaker tags/written/novels/nov_01.txt'

with open(path, 'r') as file:
    text = file.read()


In [6]:
path = 'ICE-Nigeria-txt-and-xml-files_version_Nov_03_2015/ice-nig/txt - without speaker tags/written/novels/nov_01.txt'

with open(path, 'r') as file:
    text = file.read()

import pandas as pd
# Split the text into rows
rows = text.split('\n')

# Split each row into columns
data = [row.split('\t') for row in rows]

# Convert the data to a pandas dataframe
df = pd.DataFrame(data)


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load pre-trained model and tokenizer
model_name = 'unitary/toxic-bert'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a function to classify a text
def classify_toxicity(text):
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    is_toxic = probabilities[0][1] > 0.5
    return is_toxic

# Test the function
print(classify_toxicity(df))

In [ ]:
path = 'deepcut.txt'

with open(path, 'r') as file:
    transcript = file.read()

import re

# Your transcript text
 


# Define a regular expression pattern for timestamps with hours, minutes, and seconds
timestamp_pattern = re.compile(r'\d+:\d+(:\d+)?')

# Remove timestamps from the text
cleaned_text = re.sub(timestamp_pattern, '', transcript)

# Remove empty lines
cleaned_text = '\n'.join(line for line in cleaned_text.splitlines() if line.strip())

# Save the cleaned text to a file
output_path = 'cleaned_transcript.txt'
with open(output_path, 'w') as output_file:
    output_file.write(cleaned_text)

# Print the cleaned text
#print(cleaned_text)
print(f"\nCleaned text saved to '{output_path}'.")


In [8]:
path = 'cleaned_transcript.txt'

with open(path, 'r') as file:
    text = file.read()

import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Split the text into rows
rows = text.split('\n')

# Split each row into columns
data = [row.split('\t') for row in rows]

# Convert the data to a pandas dataframe
df = pd.DataFrame(data, columns=["Text"])

# Load pre-trained model and tokenizer
model_name = 'unitary/toxic-bert'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a function to classify a text
def classify_toxicity(text):
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    toxicity_percentage = probabilities[0][1].item() * 100
    return toxicity_percentage

# Apply the function to each row in the DataFrame
df['ToxicityPercentage'] = df['Text'].apply(classify_toxicity)

# Calculate the overall percentage of toxicity for the entire document
overall_toxicity_percentage = df['ToxicityPercentage'].mean()

# Display the result
print(f"Overall Toxicity Percentage: {overall_toxicity_percentage}")


Overall Toxicity Percentage: 2.900961439768036


### This code creates a DataFrame with columns for "Text" and "Toxicity" and saves it to a CSV file named 'toxicity_results.csv'. Adjust the output_csv_path variable to specify a different file path if needed.

In [9]:
import re
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Read the cleaned transcript from file
path = 'cleaned_transcript.txt'
with open(path, 'r') as file:
    text = file.read()

# Remove timestamps and empty lines
timestamp_pattern = re.compile(r'\d+:\d+(:\d+)?')
cleaned_text = re.sub(timestamp_pattern, '', text)
cleaned_text = '\n'.join(line for line in cleaned_text.splitlines() if line.strip())

# Split the text into rows
rows = cleaned_text.split('\n')

# Load pre-trained model and tokenizer
model_name = 'unitary/toxic-bert'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a function to classify toxicity
def classify_toxicity(text):
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    toxicity_percentage = probabilities[0][1].item() * 100
    return toxicity_percentage

# Create a list to store results
toxicity_data = []

# Apply the function to each row in the DataFrame
for row in rows:
    toxicity_percentage = classify_toxicity(row)
    toxicity_data.append({"Text": row, "Toxicity": toxicity_percentage})

# Create a DataFrame from the results
df_toxicity = pd.DataFrame(toxicity_data)

# Save the DataFrame to a CSV file
output_csv_path = 'toxicity_results.csv'
df_toxicity.to_csv(output_csv_path, index=False)

# Display the DataFrame
#print(df_toxicity)
print(f"\nToxicity results saved to '{output_csv_path}'.")


                                                  Text  Toxicity
0                                            thank you  8.900988
1                                            hey there  7.190707
2                          love is a very strong thing  7.690600
3                                        my sweetheart  1.698025
4                               did I love you so much  5.644903
..                                                 ...       ...
667  until you get home don't start on  the roads t...  0.314047
668                         oh let them be let them be  0.342521
669                             to be together forever  4.373112
670                             to build this would be  7.262250
671                                             Shorts  0.385679

[672 rows x 2 columns]

Toxicity results saved to 'toxicity_results.csv'.


In [20]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.nn.functional import softmax

def detect_toxicity(text, model, tokenizer):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt")

    # Forward pass, get logits
    outputs = model(**inputs)
    logits = outputs.logits

    # Apply softmax to get probabilities
    probabilities = softmax(logits, dim=1)

    # Get the probability of the toxic class
    toxic_probability = probabilities[0, 1].item()

    return toxic_probability

# Load pretrained DistilBERT model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Example text
text = "he is a asshole"

# Detect toxicity
toxicity_probability = detect_toxicity(text, model, tokenizer)

# Print result
print(f"Toxicity Probability: {toxicity_probability}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Toxicity Probability: 0.527604877948761


In [29]:
import evaluate

toxicity = evaluate.load("toxicity", module_type="measurement")
input_texts = ["She is lovely","he is a douchebag"]
results = toxicity.compute(predictions=input_texts)
print([round(s, 4) for s in results["toxicity"]])



Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint


[0.0001, 0.8564]


In [25]:
import pandas as pd
import evaluate

def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

toxicity = evaluate.load("toxicity", module_type="measurement")

# Specify the path to your text file
file_path = 'ICE-Nigeria-txt-and-xml-files_version_Nov_03_2015/ice-nig/txt - without speaker tags/written/novels/nov_01.txt'


# Read the text from the file into a DataFrame
df = pd.DataFrame({'Text': [read_text_from_file(file_path)]})

# Compute toxicity for each text in the DataFrame
results = toxicity.compute(predictions=df['Text'].tolist())
df['Toxicity'] = [round(score, 4) for score in results["toxicity"]]

# Print the DataFrame
print(df)


Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint


                                                Text  Toxicity
0  A short story by Muhd Ibrahim Muhsin\nGwaggo c...    0.0709


In [ ]:
import os
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Folder containing the text files
folder_path = 'ICE-Nigeria-txt-and-xml-files_version_Nov_03_2015/ice-nig/txt - without speaker tags/written/novels/'

# Load pre-trained model and tokenizer
model_name = 'unitary/toxic-bert'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a function to classify toxicity
def classify_toxicity(text):
    try:
        inputs = tokenizer(text, return_tensors='pt')
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        toxicity_percentage = probabilities[0][1].item() * 100
        return toxicity_percentage
    except Exception as e:
        # Handle errors during classification
        print(f"Error classifying text: {e}")
        return None

# Create an empty DataFrame to store results
results_df = pd.DataFrame(columns=['File', 'Text', 'ToxicityPercentage'])

# Iterate through each text file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        
        # Read the content of the file
        with open(file_path, 'r') as file:
            text = file.read()

        # Classify toxicity and store results in DataFrame
        toxicity_percentage = classify_toxicity(text)
        results_df = results_df.append({'File': filename, 'Text': text, 'ToxicityPercentage': toxicity_percentage}, ignore_index=True)

# Save the results to a CSV file
results_df.to_csv('toxicity_results.csv', index=False)

# Display the DataFrame
print(results_df)


In [3]:
import pandas as pd
import evaluate
import os

def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

toxicity = evaluate.load("toxicity", module_type="measurement")

# Specify the folder path containing text files
folder_path = 'ICE-Nigeria-txt-and-xml-files_version_Nov_03_2015/ice-nig/txt - without speaker tags/written/novels/'

# Create an empty DataFrame to store results
results_df = pd.DataFrame(columns=['File', 'Text', 'Toxicity'])

# Iterate through each text file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        
        # Read the text from the file into a temporary DataFrame
        temp_df = pd.DataFrame({'Text': [read_text_from_file(file_path)]})

        # Compute toxicity for each text in the temporary DataFrame
        results = toxicity.compute(predictions=temp_df['Text'].tolist())
        toxicity_score = round(results["toxicity"][0], 4)

        # Append results to the main DataFrame
        results_df = pd.concat([results_df, pd.DataFrame({'File': [filename], 'Text': temp_df['Text'].iloc[0], 'Toxicity': [toxicity_score]})], ignore_index=True)

# Print the DataFrame
print(results_df)


Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint
/var/folders/s1/rqvf1f_d5zb106l4ym22x_6r0000gn/T/ipykernel_82570/4011217787.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({'File': [filename], 'Text': temp_df['Text'].iloc[0], 'Toxicity': [toxicity_score]})], ignore_index=True)


          File                                               Text  Toxicity
0   nov_20.txt  'That's all right.' Kehinde, too, started to u...    0.0870
1   nov_08.txt  The Fisherman of Iboko\nTen yards of the fines...    0.1971
2   nov_09.txt  "That one called Babby is a wise owl, you know...    0.0673
3   nov_19.txt  Threshold of Madness\nGOT BOTH OF YOU AT LAST!...    0.0232
4   nov_18.txt  What the Spirit Said\nOn the seventh second of...    0.0606
5   nov_01.txt  A short story by Muhd Ibrahim Muhsin\nGwaggo c...    0.0709
6   nov_15.txt  The two lay exhausted on the massive bed. The ...    0.2251
7   nov_14.txt  "You must be out of your mind!" Sade shouted a...    0.0322
8   nov_16.txt  Mazi Ogaranya, the 'Agada-gbachiri-uzo and one...    0.0298
9   nov_02.txt  I REMEMBER GETTING HOME THAT DAY AND SEEING MA...    0.1633
10  nov_03.txt  Back to the land of the animals\nThe lion was ...    0.0426
11  nov_17.txt  Chimezie and Chuby planned for the first exhib...    0.1424
12  nov_13.t

In [2]:
import pandas as pd
import evaluate
import os

def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

toxicity = evaluate.load("toxicity", module_type="measurement")

# Specify the folder path containing text files
folder_path = 'transcript_deepcut.txt'
#folder_path = 'ICE-Nigeria-txt-and-xml-files_version_Nov_03_2015/ice-nig/txt - without speaker tags/spoken all/'

# Create an empty DataFrame to store results
results_df = pd.DataFrame(columns=['File', 'Toxicity'])

# Iterate through each text file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        
        # Read the text from the file into a temporary DataFrame
        temp_df = pd.DataFrame({'Text': [read_text_from_file(file_path)]})

        # Compute toxicity for each text in the temporary DataFrame
        results = toxicity.compute(predictions=temp_df['Text'].tolist())
        toxicity_score = round(results["toxicity"][0], 4)

        # Append results to the main DataFrame
        results_df = pd.concat([results_df, pd.DataFrame({'File': [filename], 'Toxicity': [toxicity_score]})], ignore_index=True)

# Save the results to a CSV file
results_df.to_csv('toxicity_results.csv', index=False)

# Display the DataFrame
#print(results_df)


Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint


NotADirectoryError: [Errno 20] Not a directory: 'transcript_deepcut.txt'